# NDVI SMOOTHING ANALYSIS

## Import modules

In [1]:
import array
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from termcolor import colored
import time
import xarray as xr

%reload_ext Cython

## Functions

Cython functions, modified `ws2dvopt` and `ws2dvoptp` in order to get vcurve

In [50]:
%%cython

from cpython.array cimport array, clone
from libc.math cimport log, pow, sqrt
cimport numpy as np
import numpy as np

tFloat = np.double
ctypedef np.double_t dtype_t


cpdef lag1corr(np.ndarray[dtype_t] data1, np.ndarray[dtype_t] data2, double nd):
    """Calculates Lag-1 autocorrelation.

    Adapted from https://stackoverflow.com/a/29194624/5997555

    Args:
        data1: fist data series
        data2: second data series
        nd: no-data value (will be exluded from calulation)

    Returns:
        Lag-1 autocorrelation value
    """

    cdef int M, sub
    cdef double sum1, sum2, var_sum1, var_sum2, cross_sum, std1, std2, cross_mean

    M = data1.size

    sum1 = 0.
    sum2 = 0.
    sub = 0
    for i in range(M):
        if data1[i] != nd and data2[i] != nd:
            sum1 += data1[i]
            sum2 += data2[i]
        else:
            sub += 1
    mean1 = sum1 / (M-sub)
    mean2 = sum2 / (M-sub)

    var_sum1 = 0.
    var_sum2 = 0.
    cross_sum = 0.
    for i in range(M):
        if data1[i] != nd and data2[i] != nd:
            var_sum1 += (data1[i] - mean1) ** 2
            var_sum2 += (data2[i] - mean2) ** 2
            cross_sum += (data1[i] * data2[i])

    std1 = (var_sum1 / (M-sub)) ** .5
    std2 = (var_sum2 / (M-sub)) ** .5
    cross_mean = cross_sum / (M-sub)
    return (cross_mean - mean1 * mean2) / (std1 * std2)

cpdef ws2d(np.ndarray[dtype_t] y, double lmda, np.ndarray[dtype_t] w):
    cdef array dbl_array_template = array('d', [])
    cdef int i, i1, i2, m, n
    cdef array z, d, c, e

    n = y.shape[0]
    m = n - 1

    z = clone(dbl_array_template, n, zero=False)
    d = clone(dbl_array_template, n, zero=False)
    c = clone(dbl_array_template, n, zero=False)
    e = clone(dbl_array_template, n, zero=False)

    d.data.as_doubles[0] = w[0] + lmda
    c.data.as_doubles[0] = (-2 * lmda) / d.data.as_doubles[0]
    e.data.as_doubles[0] = lmda /d.data.as_doubles[0]
    z.data.as_doubles[0] = w[0] * y[0]
    d.data.as_doubles[1] = w[1] + 5 * lmda - d.data.as_doubles[0] * (c.data.as_doubles[0] * c.data.as_doubles[0])
    c.data.as_doubles[1] = (-4 * lmda - d.data.as_doubles[0] * c.data.as_doubles[0] * e.data.as_doubles[0]) / d.data.as_doubles[1]
    e.data.as_doubles[1] =  lmda / d.data.as_doubles[1]
    z.data.as_doubles[1] = w[1] * y[1] - c.data.as_doubles[0] * z.data.as_doubles[0]
    for i in range(2, m-1):
        i1 = i - 1
        i2 = i - 2
        d.data.as_doubles[i]= w[i] + 6 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
        c.data.as_doubles[i] = (-4 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1])/ d.data.as_doubles[i]
        e.data.as_doubles[i] =  lmda / d.data.as_doubles[i]
        z.data.as_doubles[i] = w[i] * y[i] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 2
    i2 = m - 3
    d.data.as_doubles[m - 1] = w[m - 1] + 5 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    c.data.as_doubles[m - 1] = (-2 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1]) / d.data.as_doubles[m - 1]
    z.data.as_doubles[m - 1] = w[m - 1] * y[m - 1] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 1
    i2 = m - 2
    d.data.as_doubles[m] = w[m] +  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    z.data.as_doubles[m] = (w[m] * y[m] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]) / d.data.as_doubles[m]
    z.data.as_doubles[m - 1] = z.data.as_doubles[m - 1] / d.data.as_doubles[m - 1] - c.data.as_doubles[m - 1] * z.data.as_doubles[m]
    for i in range(m-2, -1, -1):
        z.data.as_doubles[i] = z.data.as_doubles[i] / d.data.as_doubles[i] - c.data.as_doubles[i] * z.data.as_doubles[i + 1] - e.data.as_doubles[i] * z.data.as_doubles[i + 2]
    return z

cpdef ws2dp(np.ndarray[dtype_t] y, double lmda, np.ndarray[dtype_t] w, double p):
  """Whittaker smoother with asymmetric smoothing and fixed lambda (S).

  Args:
      y: time-series numpy array
      l: smoothing parameter lambda (S)
      w: weights numpy array
      p: "Envelope" value

  Returns:
      Smoothed time-series array z
  """
  cdef array template = array('d', [])
  cdef int m, i, j
  cdef double y_tmp, z_tmp, p1

  m = y.shape[0]
  i = 0
  j = 0
  p1 = 1-p

  template = array('d', [])
  z = clone(template, m, True)
  znew = clone(template, m, True)
  wa = clone(template, m, False)
  ww = clone(template, m, False)

  # Calculate weights

  for i in range(10):
    for j in range(m):
      y_tmp = y[j]
      z_tmp = z.data.as_doubles[j]

      if y_tmp > z_tmp:
        wa.data.as_doubles[j] = p
      else:
        wa.data.as_doubles[j] = p1
      ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

    znew[0:m] = _ws2d(y, lmda, ww)
    z_tmp = 0.0
    j = 0
    for j in range(m):
      z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

    if z_tmp == 0.0:
      break

    z[0:m]= znew[0:m]

  z[0:m] = _ws2d(y, lmda, ww)
  return z


cdef _ws2d(np.ndarray[dtype_t] y, double lmda, array[double] w):
    """Internal whittaker function for use in asymmetric smoothing.
    Args:
      y: time-series numpy array
      lmbda: lambda (s) value
      w: weights numpy array
    Returns:
        smoothed time-series array z
    """

    cdef array dbl_array_template = array('d', [])
    cdef int i, i1, i2, m, n
    cdef array z, d, c, e

    n = y.shape[0]
    m = n - 1

    z = clone(dbl_array_template, n, zero=False)
    d = clone(dbl_array_template, n, zero=False)
    c = clone(dbl_array_template, n, zero=False)
    e = clone(dbl_array_template, n, zero=False)

    d.data.as_doubles[0] = w.data.as_doubles[0] + lmda
    c.data.as_doubles[0] = (-2 * lmda) / d.data.as_doubles[0]
    e.data.as_doubles[0] = lmda /d.data.as_doubles[0]
    z.data.as_doubles[0] = w.data.as_doubles[0] * y[0]
    d.data.as_doubles[1] = w.data.as_doubles[1] + 5 * lmda - d.data.as_doubles[0] * (c.data.as_doubles[0] * c.data.as_doubles[0])
    c.data.as_doubles[1] = (-4 * lmda - d.data.as_doubles[0] * c.data.as_doubles[0] * e.data.as_doubles[0]) / d.data.as_doubles[1]
    e.data.as_doubles[1] =  lmda / d.data.as_doubles[1]
    z.data.as_doubles[1] = w.data.as_doubles[1] * y[1] - c.data.as_doubles[0] * z.data.as_doubles[0]
    for i in range(2, m-1):
        i1 = i - 1
        i2 = i - 2
        d.data.as_doubles[i]= w.data.as_doubles[i] + 6 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
        c.data.as_doubles[i] = (-4 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1])/ d.data.as_doubles[i]
        e.data.as_doubles[i] =  lmda / d.data.as_doubles[i]
        z.data.as_doubles[i] = w.data.as_doubles[i] * y[i] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 2
    i2 = m - 3
    d.data.as_doubles[m - 1] = w.data.as_doubles[m - 1] + 5 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    c.data.as_doubles[m - 1] = (-2 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1]) / d.data.as_doubles[m - 1]
    z.data.as_doubles[m - 1] = w.data.as_doubles[m - 1] * y[m - 1] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 1
    i2 = m - 2
    d.data.as_doubles[m] = w.data.as_doubles[m] +  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    z.data.as_doubles[m] = (w.data.as_doubles[m] * y[m] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]) / d.data.as_doubles[m]
    z.data.as_doubles[m - 1] = z.data.as_doubles[m - 1] / d.data.as_doubles[m - 1] - c.data.as_doubles[m - 1] * z.data.as_doubles[m]
    for i in range(m-2, -1, -1):
        z.data.as_doubles[i] = z.data.as_doubles[i] / d.data.as_doubles[i] - c.data.as_doubles[i] * z.data.as_doubles[i + 1] - e.data.as_doubles[i] * z.data.as_doubles[i + 2]
    return z


cpdef ws2doptv(np.ndarray[dtype_t] y, np.ndarray[dtype_t] w, array[double] llas):
    """Whittaker smoother with normal V-curve optimization of lambda (S).
    Args:
        y: time-series numpy array
        w: weights numpy array
        llas: array with lambda values to iterate (S-range)
    Returns:
        Smoothed time-series array z and optimized lambda (S) value lopt
    """
    cdef array template = array('d', [])
    cdef array fits, pens, diff1, lamids, v, z
    cdef int m, m1, m2, nl, nl1, lix, i, k
    cdef double w_tmp, y_tmp, z_tmp, z2, llastep, f1, f2, p1, p2, l, l1, l2, vmin, lopt

    m = y.shape[0]
    m1 = m - 1
    m2 = m - 2
    nl = len(llas)
    nl1 = nl - 1
    i = 0
    k = 0

    template = array('d', [])

    fits = clone(template, nl, True)
    pens = clone(template, nl, True)
    z = clone(template, m, False)
    diff1 = clone(template, m1, True)
    lamids = clone(template, nl1, False)
    v = clone(template, nl1, False)

    # Compute v-curve
    for lix in range(nl):
        l = pow(10,llas.data.as_doubles[lix])
        z[0:m] = ws2d(y, l, w)
        for i in range(m):
            w_tmp = w[i]
            y_tmp = y[i]
            z_tmp = z.data.as_doubles[i]
            fits.data.as_doubles[lix] += pow(w_tmp * (y_tmp - z_tmp),2) 
        fits.data.as_doubles[lix] = log(fits.data.as_doubles[lix])

        for i in range(m1):
            z_tmp = z.data.as_doubles[i]
            z2 = z.data.as_doubles[i+1]
            diff1.data.as_doubles[i] = z2 - z_tmp
        for i in range(m2):
            z_tmp = diff1.data.as_doubles[i]
            z2 = diff1.data.as_doubles[i+1]
            pens.data.as_doubles[lix] += pow(z2 - z_tmp,2)
        pens.data.as_doubles[lix] = log(pens.data.as_doubles[lix])

    # Construct v-curve
    llastep = llas[1] - llas[0]

    for i in range(nl1):
        l1 = llas.data.as_doubles[i]
        l2 = llas.data.as_doubles[i+1]
        f1 = fits.data.as_doubles[i]
        f2 = fits.data.as_doubles[i+1]
        p1 = pens.data.as_doubles[i]
        p2 = pens.data.as_doubles[i+1]
        v.data.as_doubles[i] = sqrt(pow(f2 - f1,2) + pow(p2 - p1,2)) / (log(10) * llastep)
        lamids.data.as_doubles[i] = (l1+l2) / 2

    vmin = v.data.as_doubles[k]
    for i in range(1, nl1):
        if v.data.as_doubles[i] < vmin:
            vmin = v.data.as_doubles[i]
            k = i

    lopt = pow(10, lamids.data.as_doubles[k])

    z[0:m] = ws2d(y, lopt, w)

    return z, lopt, v, lamids



cpdef ws2doptvp(np.ndarray[dtype_t] y, np.ndarray[dtype_t] w, array[double] llas, double p):
    """Whittaker smoother with asymmetric V-curve optimization of lambda (S).
    Args:
        y: time-series numpy array
        w: weights numpy array
        llas: array with lambda values to iterate (S-range)
        p: "Envelope" value
    Returns:
        Smoothed time-series array z and optimized lambda (S) value lopt
    """
    cdef array template = array('d', [])
    cdef array fits, pens, diff1, lamids, v, z
    cdef int m, m1, m2, nl, nl1, lix, i, j, k
    cdef double w_tmp, y_tmp, z_tmp, z2, llastep, fit1, fit2, pen1, pen2, l, l1, l2, vmin, lopt, p1

    m = y.shape[0]
    m1 = m - 1
    m2 = m - 2
    nl = len(llas)
    nl1 = nl - 1
    i = 0
    k = 0
    j = 0
    p1 = 1-p

    template = array('d', [])
    fits = clone(template, nl, True)
    pens = clone(template, nl, True)
    z = clone(template, m, True)
    znew = clone(template, m, True)
    diff1 = clone(template, m1, True)
    lamids = clone(template, nl1, False)
    v = clone(template, nl1, False)
    wa = clone(template, m, False)
    ww = clone(template, m, False)

    # Compute v-curve
    for lix in range(nl):
        l = pow(10,llas.data.as_doubles[lix])

        for i in range(10):
          for j in range(m):
            y_tmp = y[j]
            z_tmp = z.data.as_doubles[j]
            if y_tmp > z_tmp:
              wa.data.as_doubles[j] = p
            else:
              wa.data.as_doubles[j] = p1
            ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

          znew[0:m] = _ws2d(y, l, ww)
          z_tmp = 0.0
          j = 0
          for j in range(m):
            z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

          if z_tmp == 0.0:
            break

          z[0:m]= znew[0:m]

        for i in range(m):
            w_tmp = w[i]
            y_tmp = y[i]
            z_tmp = z.data.as_doubles[i]
            fits.data.as_doubles[lix] += pow(w_tmp * (y_tmp - z_tmp),2)
        fits.data.as_doubles[lix] = log(fits.data.as_doubles[lix])

        for i in range(m1):
            z_tmp = z.data.as_doubles[i]
            z2 = z.data.as_doubles[i+1]
            diff1.data.as_doubles[i] = z2 - z_tmp
        for i in range(m2):
            z_tmp = diff1.data.as_doubles[i]
            z2 = diff1.data.as_doubles[i+1]
            pens.data.as_doubles[lix] += pow(z2 - z_tmp,2)
        pens.data.as_doubles[lix] = log(pens.data.as_doubles[lix])

    # Construct v-curve
    llastep = llas[1] - llas[0]

    for i in range(nl1):
        l1 = llas.data.as_doubles[i]
        l2 = llas.data.as_doubles[i+1]
        fit1 = fits.data.as_doubles[i]
        fit2 = fits.data.as_doubles[i+1]
        pen1 = pens.data.as_doubles[i]
        pen2 = pens.data.as_doubles[i+1]
        v.data.as_doubles[i] = sqrt(pow(fit2 - fit1,2) + pow(pen2 - pen1,2)) / (log(10) * llastep)
        lamids.data.as_doubles[i] = (l1+l2) / 2

    vmin = v.data.as_doubles[k]
    for i in range(1, nl1):
        if v.data.as_doubles[i] < vmin:
            vmin = v.data.as_doubles[i]
            k = i

    lopt = pow(10, lamids.data.as_doubles[k])

    del z
    z = clone(template, m, True)

    for i in range(10):
      for j in range(m):
        y_tmp = y[j]
        z_tmp = z.data.as_doubles[j]

        if y_tmp > z_tmp:
          wa.data.as_doubles[j] = p
        else:
          wa.data.as_doubles[j] = p1
        ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

      znew[0:m] = _ws2d(y, lopt, ww)
      z_tmp = 0.0
      j = 0
      for j in range(m):
        z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

      if z_tmp == 0.0:
        break

      z[0:m]= znew[0:m]

    z[0:m] = _ws2d(y, lopt, ww)
    
    return z, lopt, v, lamids

Other functions

In [18]:
def daily_interpolation(rawdates: np.ndarray):
    
    #create daily date range (we add 7 days so that we don't have any problems when shifting the values later)
    pd_daily_dates = pd.date_range(start=rawdates[0],end=rawdates[-1] + datetime.timedelta(16))
    
    #convert pd data range to numpy array 
    np_daily_dates = np.array(pd_daily_dates.to_pydatetime())
    
    #convert datetime to date
    np_daily_dates = np.array([x.date() for x in np_daily_dates])
    
    return np_daily_dates

In [4]:
def fromstring(x):
    
    '''Converts string to datetime object'''
    
    try:
        d = datetime.datetime.strptime(x, '%d/%m/%Y').date()
    except:
        d = datetime.datetime.strptime(x, '%Y-%m-%d').date()
        
    return d

def fromjulian(x):
    
    '''Converts julian to datetime object'''

    return datetime.datetime.strptime(x, '%Y%j').date()

Extract time series functions

In [5]:
def ndvi_select_satellite(sat: str, MD: tuple):
    
    (MOD, MYD, MXD) = MD
    
    if (sat == 'MYD'):
        df = MYD
    elif (sat == 'MOD'):
        df = MOD
    else:
        df = MXD
        
    return df

def ndvi_extract_ts(df: pd.DataFrame, location: str, date_begin: int, date_end: int):
    
    y = df['NDVI'].loc[location].values

    dts = df['Date'].loc[location].values

    c_dts = df['Composite_date'].loc[location].values

    same_c = df['Same_composite'].loc[location].values
    
    #Crop to date range
    date_range = np.all([dts>=datetime.date(date_begin,1,1), dts<=datetime.date(date_end,12,31)], axis=0)
    y = y[date_range]
    dts = dts[date_range]
    c_dts = c_dts[date_range]
    same_c = same_c[date_range]
    
    return (y, dts, c_dts, same_c)

Smoothing functions

In [49]:
def ndvi_smoothing_A0(y: np.ndarray, dts: np.ndarray, pvalue: float, nopval: bool, lrange: np.ndarray, nd: int):
    
     # create weights
    w = np.array((y!=nd)*1,dtype='double')

    # apply whittaker filter with V-curve
    if (nopval):
        z, lopt, vcurve, l = ws2doptv(y, w, array.array('d',lrange))
    else:
        z, lopt, vcurve, l = ws2doptvp(y, w, array.array('d',lrange), pvalue)
    
    return (z, lopt, vcurve, l)


    
def ndvi_smoothing_A1(y: np.ndarray, dts: np.ndarray, pvalue: float, nopval: bool, lrange: np.ndarray, nd: int):

    # create weights
    w = np.array((y!=nd)*1,dtype='double')

    # apply whittaker filter with V-curve
    if (nopval):
        z, lopt, vcurve, l = ws2doptv(y, w, array.array('d',lrange))
    else:
        z, lopt, vcurve, l = ws2doptvp(y, w, array.array('d',lrange), pvalue)

    # Temporal interpolation
    daily = daily_interpolation(dts)
    dvec = np.full(len(daily), nd, dtype='double')

    # shift observations to midpoint of acquisition (these positions are set to 0 instead of nodata)
    mid_point = 16/2
    for d in dts:
        dl = daily.tolist()
        dvec[dl.index(d + datetime.timedelta(mid_point))] = 0

    # place des filtered values in the midpoints
    dvec[ dvec != nd ] = z

    # recreate weights
    w = np.array((dvec != nd) * 1,dtype='double')

    #refilter with low lanmbda
    z =  ws2d(dvec, 0.0001, w)
    
    return (z, lopt, vcurve, l, daily)


def ndvi_smoothing_A2(y: np.ndarray, dts: np.ndarray, pvalue: float, nopval: bool, lrange: np.ndarray, nd: int):
    
    # Temporal interpolation
    daily = daily_interpolation(dts)
    dvec = np.full(len(daily), nd, dtype='double')

    mid_point = 16/2
    for d in dts:
        dl = daily.tolist()
        dvec[dl.index(d + datetime.timedelta(mid_point))] = 0

    # place des values in the right place
    dvec[ dvec != nd ] = y

    # create weights
    w = np.array((dvec!=nd)*1,dtype='double')

    # apply whittaker filter with V-curve
    if (nopval):
        z, lopt, vcurve, l = ws2doptv(dvec, w, array.array('d',lrange))
    else:
        z, lopt, vcurve, l = ws2doptvp(dvec, w, array.array('d',lrange), pvalue)        
        
    return (z, lopt, vcurve, l)


def ndvi_smoothing_A3(y: np.ndarray, dts: np.ndarray, same_c: np.ndarray, c_dts: np.ndarray, pvalue: float, nopval: bool, lrange, nd: int):
    
    #remove the values with the same day composite
    y_unique = np.delete(y, np.argwhere(same_c))
    c_dts = np.delete(c_dts, np.argwhere(same_c))

    # Temporal interpolation
    daily = daily_interpolation(dts)
    dvec = np.full(len(daily), nd, dtype='double')


    for d in c_dts:
        dl = daily.tolist()
        dvec[dl.index(d)] = 0

    # place des values in the right place
    dvec[ dvec != nd ] = y_unique

    # create weights
    w = np.array((dvec!=nd)*1,dtype='double')

    # apply whittaker filter with V-curve
    if (nopval):
        z, lopt, vcurve, l = ws2doptv(dvec, w, array.array('d',lrange))
    else:
        z, lopt, vcurve, l = ws2doptvp(dvec, w, array.array('d',lrange), pvalue)
        
    return (z, lopt, vcurve, l)

Print and Plot functions

In [44]:
def ndvi_print_info(location: str, latlon: dict, lagCorr1: float):
    
    print('\033[1m' + 'Selected Point: ', location, '\033[0m')
    print('(Lat, Lon) =', latlon[location] )
    print('\n')
    
    print('LagCorr: ', round(lagCorr1,3))
    print('\n')
    
    
    
    
def ndvi_plot_all(A0: bool, A1: bool, A2: bool, A3: bool,
                  dts: np.ndarray, y: np.ndarray, daily: np.ndarray, 
                  z0: np.ndarray, z1: np.ndarray, z2: np.ndarray, z3: np.ndarray, 
                  nd: int, 
                  yauto: bool, ylimits: tuple):
    
    #replace nd by nan
    ynan = y.copy()
    ynan[ynan == nd] = np.nan
    
    plt.figure(figsize=(20,5))
    
    A = [A0, A1, A2, A3]
    xA = [dts, daily, daily, daily]
    z = [z0, z1, z2, z3]
    col = ['b', 'g', 'r', 'orange']
    label = ['A0', 'A1', 'A2', 'A3']
    leg = []
    
    for i,a in enumerate(A):
        if a:
            plt.plot(xA[i], z[i], color = col[i], alpha = 0.5)
            leg.append(label[i])

    leg.append('raw values')
    plt.plot(dts, ynan, color = 'grey', marker = 'o', alpha = 0.5)
    
    if not(yauto):
        plt.ylim(ylimits)
    
    plt.xlabel('Date', fontsize=15)
    plt.ylabel('NDVI', fontsize=15)
    plt.legend(leg, fontsize=17, loc = 'lower right')
    plt.show()


    
def ndvi_plot_vcurve(A0: bool, A1: bool, A2: bool, A3: bool,
                     l0: np.ndarray, l1: np.ndarray, l2: np.ndarray, l3: np.ndarray, 
                     vcurve0: np.ndarray, vcurve1: np.ndarray, vcurve2: np.ndarray, vcurve3: np.ndarray, 
                     lopt0: float, lopt1: float, lopt2: float, lopt3: float):
    
    plt.figure(figsize=(20,10))
    
    A = [A0, A1, A2, A3]
    xA = [l0, l1, l2, l3]
    v = [vcurve0, vcurve1, vcurve2, vcurve3]
    lopt = [lopt0, lopt1, lopt2, lopt3]
    col = ['blue', 'green', 'red', 'orange']
    leg = []
    
    for i,a in enumerate(A):
        if a:
            plt.plot(xA[i], v[i], color = col[i], alpha = 0.5, marker = 'o')
            leg.append('lopt: ' + str(round(np.log10(lopt[i]),2)))
            
    for i,a in enumerate(A):
        if a:
            plt.axvline(x = np.log10(lopt[i]), ls = '--', color = col[i])

    plt.xlabel('log10(l)', fontsize=15)
    plt.ylabel('V', fontsize=15)
    plt.title('V-curves', fontsize=23)
    plt.legend(leg, fontsize=17, loc = 'lower right')
    plt.show()
            
    
def ndvi_plot_year(A0: bool, A1: bool, A2: bool, A3: bool,
                   year: float, month: int, 
                   lta_mean: dict, lta_std: dict, 
                   daily: np.ndarray, 
                   z0: np.ndarray, z1: np.ndarray, z2: np.ndarray, z3: np.ndarray, 
                   z0_lta: np.ndarray, z1_lta: np.ndarray, z2_lta: np.ndarray, 
                   dts: np.ndarray, y: np.ndarray, 
                   nd: int, 
                   yauto: bool, ylimits: tuple,
                   satellite: str):

    
    #replace nd by nan
    ynan = y.copy()
    ynan[ynan == nd] = np.nan
    
    
    fig, axs = plt.subplots(1,2, figsize=(20, 10))

    # cropping daily data to year
    year_index = np.all([daily>=datetime.date(year,month,1), daily<datetime.date(year+1,month,1)], axis=0)

    year_daily = daily[year_index]
    year_z1 = np.array(z1)[year_index]
    year_z2 = np.array(z2)[year_index]
    year_z3 = np.array(z3)[year_index]
    year_z1_lta = np.array(z1_lta)[year_index]
    year_z2_lta = np.array(z2_lta)[year_index]

    # cropping raw data to year
    year_index2 = np.all([dts>=datetime.date(year,month,1), dts<datetime.date(year+1,month,1)], axis=0)

    year_dts = dts[year_index2]
    year_ynan = ynan[year_index2]
    year_z0 = np.array(z0)[year_index2]
    year_z0_lta = np.array(z0_lta)[year_index2]
    year_lta_mean = []
    year_lta_std = []
    for dt in year_dts:
        year_lta_mean.append(lta_mean[(dt.day, dt.month)])
        year_lta_std.append(lta_std[(dt.day, dt.month)])
        
    
    
    #PLOTTING THE YEARLY DATA    
    A = [A0, A1, A2, A3]
    xA = [year_dts, year_daily, year_daily, year_daily]
    z = [year_z0, year_z1, year_z2, year_z3]
    col = ['b', 'g', 'r', 'orange']
    label = ['A0', 'A1', 'A2', 'A3']
    leg = []
    
    for i,a in enumerate(A):
        if a:
            axs[0].plot(xA[i], z[i], color = col[i], alpha = 0.5)
            leg.append(label[i])

    leg.append('raw values')
    axs[0].plot(year_dts, year_ynan, color = 'grey', marker = 'o', alpha = 0.5)
    
    #Color MOD and MYD points if MXD data
    if (satellite == 'MXD'):
        len_date = len(dts[dts < datetime.date(year,month,1)])
        if (len_date % 2) == 0:
            first = 'MYD'
            second = 'MOD'
        else:
            first = 'MOD'
            second = 'MYD'
                
        axs[0].plot(year_dts[0::2], year_ynan[0::2], 'ks')
        axs[0].plot(year_dts[1::2], year_ynan[1::2], 'k^')
        leg.append(first)
        leg.append(second)
    
    if not(yauto):
        axs[0].set_ylim(ylimits)

    axs[0].set_xlabel('Date', fontsize=15)
    axs[0].set_ylabel('NDVI', fontsize=15)
    axs[0].legend(leg, fontsize=15, loc = 'lower right')
    axs[0].set_title('Year ' + str(year), fontsize = 20)
    
    
    #PLOTTING THE LTA
    
    A = [A0, A1, A2]
    xA = [year_dts, year_daily, year_daily]
    z = [year_z0_lta, year_z1_lta, year_z2_lta]
    col = ['b', 'g', 'r']
    label = ['A0 (lta)', 'A1 (lta)', 'A2 (lta)']
    leg = []
    
    for i,a in enumerate(A):
        if a:
            axs[1].plot(xA[i], z[i], color = col[i], alpha = 0.5)
            leg.append(label[i])

    axs[1].plot(year_dts, year_lta_mean, color = 'grey', marker = 'o', alpha = 0.5)
    axs[1].fill_between(year_dts, np.array(year_lta_mean)-2*np.array(year_lta_std), np.array(year_lta_mean)+2*np.array(year_lta_std), color = 'whitesmoke')
    axs[1].fill_between(year_dts, np.array(year_lta_mean)-np.array(year_lta_std), np.array(year_lta_mean)+np.array(year_lta_std), color = 'lightgrey')
    
    leg.append('raw values')
    leg.append('+- 2sd')
    leg.append('+-1sd')
    
    if not(yauto):
        axs[1].set_ylim(ylimits)

    axs[1].set_xlabel('Date', fontsize=15)
    axs[1].set_ylabel('LTA NDVI', fontsize=15)
    axs[1].legend(leg, fontsize=15, loc = 'lower right')
    axs[1].set_title('Long Term Averages ' + str(year), fontsize = 20)
    

LTA

In [8]:
def ndvi_lta_dict(y: np.ndarray, dts: np.ndarray, nd: float):
    
    dict_index = set((dt.day, dt.month) for dt in dts)

    # initialize the dicts
    lta_mean = {}
    lta_std = {}

    for dt in dict_index:

        lta_date = [] 

        for ix, date_sel in enumerate(dts):
            
            if (date_sel.month == dt[1]) & (date_sel.day == dt[0]):     
                if (y[ix] != nd):
                    lta_date.append(y[ix])


        # add to LTA dict
        lta_mean[dt] = np.mean(lta_date)
        lta_std[dt] = np.std(lta_date)
        
    return lta_mean, lta_std

Main function

In [46]:
def ndvi_main(A0: bool, A1: bool, A2: bool, A3: bool,
              MD: tuple, location: str, latlon: dict, satellite: str, 
              pvalue:float, nopval: bool, 
              lrange1: tuple, lrange2: tuple, step: float, 
              nd: int, 
              date_begin: int, date_end: int, 
              year: int, month: int,
              yauto: bool, ylimits: tuple):
    
    lrange1 = np.arange(lrange1[0],lrange1[1], step)
    lrange2 = np.arange(lrange2[0],lrange2[1], step)
    
    #extract time series
    df = ndvi_select_satellite(satellite, MD)
    (y, dts, c_dts, same_c) = ndvi_extract_ts(df, location, date_begin, date_end)
    
    #LTA
    lta_mean, lta_std = ndvi_lta_dict(y, dts, nd)
    
    #smoothing
    (z0, lopt0, vcurve0, l0) = ndvi_smoothing_A0(y, dts, pvalue, nopval, lrange1, nd)
    (z1, lopt1, vcurve1, l1, daily) = ndvi_smoothing_A1(y, dts, pvalue, nopval, lrange1, nd)
    (z2, lopt2, vcurve2, l2) = ndvi_smoothing_A2(y, dts, pvalue, nopval, lrange2, nd)
    (z3, lopt3, vcurve3, l3) = ndvi_smoothing_A3(y, dts, same_c, c_dts, pvalue, nopval, lrange2, nd)
    
    #smoothing LTA data
    # no daily date for LTA
    lta_mean_list = []
    for dt in dts:
        lta_mean_list.append(lta_mean[(dt.day, dt.month)])
    (z0_lta, lopt0_lta, vcurve0_lta, l0_lta) = ndvi_smoothing_A0(np.array(lta_mean_list), dts, pvalue, nopval, lrange1, nd)
    (z1_lta, lopt1_lta, vcurve1_lta, l1_lta, daily_lta) = ndvi_smoothing_A1(np.array(lta_mean_list), dts, pvalue, nopval, lrange1, nd)
    (z2_lta, lopt2_lta, vcurve2_lta, l2_lta) = ndvi_smoothing_A2(np.array(lta_mean_list), dts, pvalue, nopval, lrange2, nd)
    
    #lagCorr
    lagCorr1 = lag1corr(np.array(y[0:len(y)-1]), np.array(y[1:]), nd)
    
    #Prints and plots
    ndvi_print_info(location, latlon, lagCorr1)
    ndvi_plot_all(A0, A1, A2, A3, dts, y, daily, z0, z1, z2, z3, nd, yauto, ylimits)
    ndvi_plot_year(A0, A1, A2, A3, year, month, lta_mean, lta_std, daily, z0, z1, z2, z3, z0_lta, z1_lta, z2_lta, dts, y, nd, yauto, ylimits, satellite)
    ndvi_plot_vcurve(A0, A1, A2, A3, l0, l1, l2, l3, vcurve0, vcurve1, vcurve2, vcurve3, lopt0, lopt1, lopt2, lopt3)
    

## Loading the data

In [10]:
ndvi_nd = -3000

In [11]:
# loading ndvi_MOD data from csv
ndvi_MOD = pd.read_csv(
    'data/MOD13A2-MOD13A2-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MOD13A2_006__1_km_16_days_NDVI', 
               'MOD13A2_006__1_km_16_days_composite_day_of_the_year'],
    dtype = {'MOD13A2_006__1_km_16_days_composite_day_of_the_year': int})


#renaming the columns
ndvi_MOD = ndvi_MOD.rename(columns={"MOD13A2_006__1_km_16_days_NDVI": "NDVI", 
                        "MOD13A2_006__1_km_16_days_composite_day_of_the_year": "Composite_date"})


# Convert string Date to datetime.date
ndvi_MOD['Date'] = ndvi_MOD['Date'].apply(fromstring)


# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(ndvi_MOD)):
    
    d = ndvi_MOD['Date'][i]
    c = ndvi_MOD['Composite_date'][i]
    
    if c != -1:    
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(False)
    elif (compo[i]==compo[i-1]):
        same_compo.append(True)
    else:
        same_compo.append(False)
            
ndvi_MOD['Composite_date'] = compo   
ndvi_MOD['Same_composite'] = same_compo 

In [12]:
# loading MYD data from csv
ndvi_MYD = pd.read_csv(
    'data/MYD13A2-MYD13A2-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MYD13A2_006__1_km_16_days_NDVI', 
               'MYD13A2_006__1_km_16_days_composite_day_of_the_year'],
    dtype = {'MYD13A2_006__1_km_16_days_composite_day_of_the_year': int})


#renaming the columns
ndvi_MYD = ndvi_MYD.rename(columns={"MYD13A2_006__1_km_16_days_NDVI": "NDVI", 
                        "MYD13A2_006__1_km_16_days_composite_day_of_the_year": "Composite_date"})


# Convert string Date to datetime.date
ndvi_MYD['Date'] = ndvi_MYD['Date'].apply(fromstring)


# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(ndvi_MYD)):
    
    d = ndvi_MYD['Date'][i]
    c = ndvi_MYD['Composite_date'][i]
    
    if c != -1:    
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(False)
    elif (compo[i]==compo[i-1]):
        same_compo.append(True)
    else:
        same_compo.append(False)
            
ndvi_MYD['Composite_date'] = compo   
ndvi_MYD['Same_composite'] = same_compo 

In [13]:
#creating MXD

#changing the index to numbers
number_index1 = pd.Index(range(0,2*len(ndvi_MYD),2))
number_index2 = pd.Index(range(1,2*len(ndvi_MYD)+1,2))

ndvi_MYD_nb = ndvi_MYD.set_index(number_index1)
ndvi_MYD_nb['ID'] = ndvi_MYD.index
ndvi_MOD_nb = ndvi_MOD.set_index(number_index2)
ndvi_MOD_nb['ID'] = ndvi_MOD.index


#concatenating and resetting ID as index
ndvi_MXD = pd.concat([ndvi_MYD_nb, ndvi_MOD_nb]).sort_index()
ndvi_MXD = ndvi_MXD.set_index('ID')

#re-run same_composite
same_compo = []
for i in range(len(ndvi_MXD)):
    if (i==0 or i==1):
        same_compo.append(False)
    elif (ndvi_MXD['Composite_date'][i]==ndvi_MXD['Composite_date'][i-1] or ndvi_MXD['Composite_date'][i]==ndvi_MXD['Composite_date'][i-2]):
        same_compo.append(True)
    else:
        same_compo.append(False)
              
ndvi_MXD['Same_composite'] = same_compo 

In [10]:
#Creating lat lon dictionnary

latlon = {}

pd_latlon = pd.read_csv(
    'data/MYD13A2-MYD13A2-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Latitude', 
               'Longitude'])

dict_index = set(pd_latlon.index)

for location in dict_index:
    latlon[location] = (round(pd_latlon.loc[location]['Latitude'][0],3), round(pd_latlon.loc[location]['Longitude'][0],3))